# 7 УРОК ПОДЗАПРОСЫ

## О порядке выполнения запросов в PostgreSQL
К текущему уроку мы уже успели познакомиться с основными операторами, которые составляют «скелет» стандартного SQL-запроса: SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY и LIMIT.

Мы уже знаем, что порядок их написания в запросе следующий:

SELECT -- перечисление полей результирующей таблицы
FROM -- указание источника данных
WHERE -- фильтрация данных
GROUP BY -- группировка данных
HAVING -- фильтрация данных после группировки
ORDER BY -- сортировка результирующей таблицы
LIMIT -- ограничение количества выводимых записей


Тем не менее важно понимать, что порядок выполнения операторов в СУБД несколько отличается от порядка их написания в запросе. В упрощённом виде порядок выполнения запроса в PostgreSQL такой:

FROM -- указание источника данных
WHERE -- фильтрация данных
GROUP BY -- группировка данных
HAVING -- фильтрация данных после группировки
SELECT -- перечисление полей результирующей таблицы
ORDER BY -- сортировка результирующей таблицы
LIMIT -- ограничение количества выводимых записей


Таким образом:

Сначала с помощью FROM определяется таблица.
Затем в соответствии с указанным в WHERE условием из этой таблицы отбираются записи.
Потом выбранные данные группируются и агрегируются с помощью GROUP BY.
Далее из агрегированных записей отбираются те, которые удовлетворяют условию в HAVING.
Только после этого в соответствии с указанными в SELECT инструкциями формируется результирующая таблица — производятся все необходимые вычисления, присваиваются новые имена и т.д.
Затем результирующая таблица сортируется в соответствии с ORDER BY.
И наконец срабатывает ограничение на количество строк, указанное в LIMIT.
На самом деле это очень важная информация, которую следует держать в голове при составлении любых SQL-запросов. 

Возможно, вы уже столкнулись с ошибками, когда пытались в блоке WHERE использовать фильтрацию по новым полям, рассчитанным в SELECT. Такая операция не соответствует порядку выполнения операторов, и поэтому так делать нельзя. Также вы могли попасть в ситуацию, когда при фильтрации в HAVING у вас не принимался алиас колонки, рассчитанной в SELECT. Это происходило по той же причине — PostreSQL просто ещё не знал о том, что вы присвоили колонке новое имя.

И ещё один важный совет. Обратите внимание, что фильтрацию данных по неагрегированным значениям можно делать как в блоке WHERE, так и в блоке HAVING. Внимательно посмотрите на следующие запросы:

SELECT sex, COUNT(user_id)
FROM users
WHERE sex != 'male'
GROUP BY sex


SELECT sex, COUNT(user_id)
FROM users
GROUP BY sex
HAVING sex != 'male'


Их результат будет одинаковым (можете убедиться в этом сами).

Однако делать фильтрацию по неагрегированным данным рекомендуется именно в блоке WHERE, т.е. заранее. В таком случае вы ещё до группировки убираете из расчётов ненужные вам данные и таким образом не расходуете вычислительные ресурсы на подсчёт значений, которые всё равно будут отфильтрованы вами позже.

Это важный момент, касающийся оптимизации SQL-запросов, поэтому рекомендуем вам принять во внимание информацию, представленную на этом шаге.

## Задача 1.
Итак, на лекции мы познакомились с подзапросами (вложенными запросами) и поняли, что в целом их синтаксис ничем не отличается от синтаксиса обычных запросов, которые мы составляли ранее. Иными словами, подзапрос — это всего лишь запрос внутри другого запроса.

Подзапросы могут применяться в следующих частях основного запроса:

в операторе FROM;
в операторе SELECT (если запрос возвращает один столбец с одним значением);
в операторах WHERE и HAVING (если запрос возвращает один столбец с одним или несколькими значениями).
Но давайте обо всём по порядку.

Прежде всего важно понять, что к результату выполнения подзапроса можно обращаться так же, как и к таблицам в базе данных, т.е. использовать их в блоке FROM вместо имеющихся таблиц:

SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM table
) AS subquery_1


В рамках данного запроса сначала будет выполнен подзапрос, который отберёт колонки column_1 и column_2 из таблицы table, а затем уже из образовавшейся таблицы основной запрос выберет колонку column_1.

Важный момент: при использовании подзапроса в блоке FROM сформированной на основе подзапроса таблице необходимо присвоить какой-нибудь алиас, иначе основной запрос не сработает. В примере выше мы обозначили результат подзапроса как subquery_1.

Кроме того, уровней вложенности может быть несколько:

SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM (
        SELECT column_1, column_2, column_3
        FROM table
    ) AS subquery_1
) AS subquery_2


В данном случае последовательность работы запроса такая: сначала будет выполнен подзапрос, возвращающий результат subquery_1, затем подзапрос, возвращающий результат subquery_2, и только потом в результате основного подзапроса попадёт колонка column_1. В результате получается что-то похожее на матрёшку, при этом к основной таблице table обращается только самый первый подзапрос subquery_1.

Приведённый пример довольно условный и на практике колонки таким образом отбирать не стоит, но общую идею он должен передавать.

Понимание того, как работают подзапросы в блоке FROM, нам особенно пригодится, когда мы будем учиться объединять разные таблицы в следующем уроке.

А сейчас давайте решим простую задачу.

Задание:

Используя данные из таблицы user_actions, рассчитайте среднее число заказов всех пользователей нашего сервиса. Для этого сначала в подзапросе посчитайте, сколько заказов сделал каждый пользователь, а затем обратитесь к результату подзапроса в блоке FROM и уже в основном запросе усредните количество заказов по всем пользователям. Полученное среднее число заказов всех пользователей округлите до двух знаков после запятой. Колонку с этим значением назовите orders_avg.

Поле в результирующей таблице: orders_avg

Пояснение:

К колонкам из подзапроса можно применять агрегирующие функции — так же, как если бы мы обращались к колонкам исходных таблиц.

##### ОТВЕТ:

In [ ]:
SELECT
  ROUND(AVG(count_orders),2) AS orders_avg
FROM
  (
    SELECT
      user_id,
      COUNT(DISTINCT order_id) AS count_orders
    FROM
      user_actions
    GROUP BY
      user_id
  ) AS count_all

## Задача 2.
Решая предыдущую задачу, вы могли задаться вопросами: а что если один и тот же подзапрос будет использоваться в нескольких частях основного запроса? неужели каждый раз придётся дублировать один и тот же подзапрос? а что если к тому же уровней вложенности будет несколько? не получится ли тогда слишком сложный и громоздкий запрос, который будет сложно читать?

Для таких случаев в SQL предусмотрен оператор WITH, который позволяет создавать так называемые табличные выражения (CTE, common table expressions) — временные таблицы, существующие только для одного запроса. Их основное предназначение заключается в разбиении сложных запросов на несколько частей.

Табличные выражения создаются так:

WITH subquery_1 AS (
    SELECT column_1, column_2
    FROM table
    )

SELECT column_1
FROM subquery_1 


Сравните запрос выше с результатом запроса из прошлого шага:

SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM table
) AS subquery_1


Оператор WITH может содержать несколько табличных выражений, причём к указанным ранее выражениям можно обращаться в последующих выражениях:

WITH subquery_1 AS (
    SELECT column_1, column_2, column_3
    FROM table
    ),
     subquery_2 AS (
    SELECT column_1, column_2
    FROM subquery_1
    )

SELECT column_1
FROM subquery_2


Можете снова сравнить запрос выше с запросом из прошлого шага:

SELECT column_1
FROM (
    SELECT column_1, column_2
    FROM (
        SELECT column_1, column_2, column_3
        FROM table
    ) AS subquery_1
) AS subquery_2


Использовать в своих запросах оператор WITH или нет — решать вам, но в некоторых случаях он может упростить работу с кодом запроса.

На заметку:

Подробнее про WITH и табличные выражения можно почитать здесь.

Задание:

Повторите запрос из предыдущего задания, но теперь вместо подзапроса используйте оператор WITH и табличное выражение. Условия задачи те же.

Поле в результирующей таблице: orders_avg



##### ОТВЕТ:

In [ ]:
WITH count_all AS (
  SELECT
    user_id,
    COUNT(DISTINCT order_id) AS count_orders
  FROM
    user_actions
  GROUP BY
    user_id
)
SELECT
  ROUND(AVG(count_orders), 2) AS orders_avg
FROM
  count_all

## Задача 3.
Ещё одно важное направление применения подзапросов — создание более продвинутых условных выражений в операторах WHERE и HAVING. Но поскольку и в том, и в другом случае синтаксис и назначение подзапросов примерно одинаковые, в этом уроке мы будем рассматривать всё на примере подзапросов в WHERE.

В первую очередь важно понять, что подзапрос, возвращающий одно значение, может использоваться как обычное значение совместно с операторами сравнения.

Представьте, что нам нужно сравнить значения в каком-то столбце с максимальным, минимальным или средним значением в этом же столбце. Сделать это в рамках одного запроса не получится, так как агрегирующие функции нельзя использовать в блоке WHERE.

Например, следующий запрос работать не будет (база данных сообщит об ошибке):

SELECT column
FROM table
WHERE column = MAX(column) 


В то же время такой запрос сработает, так как подзапрос выполнится первым и вернёт одно значение:

SELECT column
FROM table
WHERE column = (SELECT MAX(column) FROM table) 


В результате выполнения такого запроса мы получим все значения в колонке column, равные максимальному значению в этой колонке.

Задание:

Выведите из таблицы products информацию о всех товарах кроме самого дешёвого. Результат отсортируйте по убыванию id товара.

Поля в результирующей таблице: product_id, name, price



##### ОТВЕТ:

In [ ]:
SELECT
  product_id,
  name,
  price
FROM
  products
WHERE
  price != (
    SELECT
      MIN(price)
    FROM
      products
  )
ORDER BY
  1 DESC

## Задача 4.
В блоке WHERE результаты подзапросов, возвращающих одно значение, также можно использовать в арифметических операциях:

SELECT column
FROM table
WHERE column = (SELECT MAX(column) FROM table) - 100


Задание:

Выведите информацию о товарах в таблице products, цена на которые превышает среднюю цену всех товаров на 20 рублей и более. Результат отсортируйте по убыванию id товара.

Поля в результирующей таблице: product_id, name, price

##### ОТВЕТ:

In [ ]:
SELECT
  product_id,
  name,
  price
FROM
  products
WHERE
  price > (
    SELECT
      AVG(price)
    FROM
      products
  ) + 20
ORDER BY
  1 DESC

## Задача 5.
В каких ещё случаях нам может пригодиться подзапрос в оператореWHERE?

Давайте представим, что нам нужно провести какие-нибудь расчёты за последние N дней — скажем, за последнюю неделю. Не будем же мы вручную отсчитывать 7 дней от последней даты в нашей таблице? Разумеется, не будем, так как последняя дата, к тому же, может со временем измениться, когда к нам поступят новые данные. Каждый раз писать новый запрос и считать дату вручную — занятие не для нас.

Для решения такой задачи мы можем сначала с помощью подзапроса вычислить последнюю дату в наших данных, а затем отложить от неё неделю.

Чтобы отложить от даты или прибавить к ней какой-то промежуток времени, можно использовать несложные арифметические операции с датами. Например, от текущей даты можно отнять какой-то промежуток INTERVAL:

SELECT NOW() - INTERVAL '1 year 2 months 1 week'

Результат:
10/10/21 19:32


Кстати, NOW() — полезная функция, которая позволяет получать текущую дату и время (в вашем случае она будет другой):

SELECT NOW()

Результат:
17/12/22 19:32


На заметку:

С другими примерами работы с INTERVAL и арифметическими операциями с датами можно ознакомиться здесь.

Про функцию NOW() можно дополнительно почитать тут.

Задание:

Посчитайте количество уникальных клиентов в таблице user_actions, сделавших за последнюю неделю хотя бы один заказ. Полученную колонку со значением назовите users_count. В качестве текущей даты, от которой откладывать неделю, используйте последнюю дату в той же таблице user_actions.

Поле в результирующей таблице: users_count

##### ОТВЕТ:

In [ ]:
SELECT
  COUNT(DISTINCT user_id) AS users_count
FROM
  user_actions
WHERE
  action = 'create_order'
  AND time > (
    SELECT
      MAX(time) - INTERVAL '1 week'
    FROM
      user_actions
  )

## Задача 6.
Получается, что в одном из прошлых уроков мы неправильно считали возраст курьеров. Действительно, в качестве первой даты в функции AGE() мы использовали текущую дату, а не последнюю дату в наших данных. Давайте исправим это досадное недоразумение, ведь теперь у нас для этого есть все необходимые знания.

Задание:

С помощью функции AGE() и агрегирующей функции снова рассчитайте возраст самого молодого курьера мужского пола в таблице couriers, но в этот раз в качестве первой даты используйте последнюю дату из таблицы courier_actions. Чтобы получилась именно дата, перед применением функции AGE() переведите посчитанную последнюю дату в формат DATE, как мы делали в этом задании. Возраст курьера измерьте количеством лет, месяцев и дней и переведите его в тип VARCHAR. Полученную колонку со значением возраста назовите min_age.

Поле в результирующей таблице: min_age

Пояснение:

В этой задаче результат подзапроса выступает в качестве аргумента функции. Чтобы весь запрос выглядел компактнее, для приведения данных к другому типу можно использовать формат записи с двумя двоеточиями — ::.

Также обратите внимание, что для получения необходимого результата мы обращаемся к разным таблицам в рамках одного общего запроса — так делать тоже можно. 

##### ОТВЕТ:

In [ ]:
# Чужой
SELECT
  MIN(AGE((
  SELECT DATE(MAX(time))
FROM courier_actions), birth_date)) :: VARCHAR AS min_age
FROM
  couriers
WHERE
  sex LIKE 'male' AND birth_date IS NOT NULL

In [ ]:
SELECT
  AGE(
    (
      SELECT
        DATE(MAX(time))
      FROM
        courier_actions
    ),
    MAX(birth_date)
  ) :: VARCHAR AS min_age
FROM
  couriers
WHERE
  sex = 'male'

In [ ]:
# Тоже МОЕ
WITH real_date AS (
  SELECT
    DATE(MAX(time))
  FROM
    courier_actions
)
SELECT
  AGE(
    (
      SELECT
        *
      FROM
        real_date
    ),
    MAX(birth_date)
  ) :: VARCHAR AS min_age
FROM
  couriers
WHERE
  sex = 'male'


## Задача 7.
Подзапрос, возвращающий несколько значений, может использоваться в блоке WHERE совместно с оператором IN — например, когда нам нужно проверить, совпадает ли значение в столбце с одним из значений из определённого множества, полученного в результате выполнения подзапроса:

SELECT column_1
FROM table_1
WHERE column_1 IN (SELECT column_2 FROM table_2) 


При этом запрос выше будет равносилен запросу с табличным выражением:

WITH subquery AS (
    SELECT column_2
    FROM table_2
    )

SELECT column_1
FROM table_1
WHERE column_1 IN (SELECT * FROM subquery) 


Обратите внимание, что при использовании в операторе WHERE табличного выражения обратиться просто к его имени нельзя — необходимо предварительно выбрать все его записи. т.е. написать подзапрос. При этом в табличном выражении должен быть всего один столбец, иначе база данных вернёт ошибку.

Кроме того, в табличном выражении можно хранить всего одно значение (например, результат агрегации) и аналогичным образом вызывать его в операторе WHERE как переменную:

WITH subquery AS (
    SELECT MAX(column_2)
    FROM table_2
    )

SELECT column_1
FROM table_1
WHERE column_1 = (SELECT * FROM subquery) 


Давайте рассмотрим следующий пример.

Из наших данных довольно легко отобрать отменённые заказы — достаточно указать в WHERE нужный фильтр. Но как отобрать созданные, но неотменённые заказы? Это как раз можно сделать с помощью подзапроса или табличного выражения.

Задание:

Из таблицы user_actions с помощью подзапроса или табличного выражения отберите все заказы, которые не были отменены пользователями. Выведите колонку с id этих заказов. Результат запроса отсортируйте по возрастанию id заказа.

Поле в результирующей таблице: order_id

##### ОТВЕТ:

In [ ]:
SELECT order_id
FROM user_actions
WHERE order_id NOT IN
(SELECT order_id
FROM user_actions
WHERE action = 'cancel_order')
ORDER BY 1
  

## Задача 8.
Как было отмечено в первом шаге, вложенный запрос может располагаться и после оператора SELECT. Однако результатом подзапроса в таком случае может быть только одно значение — например, результат применения агрегирующей функции к какой-либо колонке:

SELECT column_1, (SELECT MAX(column_1) FROM table) AS max_column_1
FROM table


В данном случае из таблицы table будет выбрана колонка column_1, и напротив каждого значения в этой колонке будет выведен результат выполнения вложенного запроса, т.е. максимальное значение в этой колонке. При этом давать алиас результату подзапроса не обязательно.

Также результаты подзапросов в блоке SELECT можно использовать в вычислениях:

SELECT column_1, (SELECT MAX(column_1) FROM table) - 100 AS column_2
FROM table


Задание:

Используя данные из таблицы user_actions, рассчитайте, сколько заказов сделал каждый пользователь и отразите это в столбце orders_count. В отдельном столбце orders_avg напротив каждого пользователя укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой. Также для каждого пользователя посчитайте отклонение числа заказов от среднего значения. Отклонение считайте так: число заказов «минус» округлённое среднее значение. Колонку с отклонением назовите orders_diff. Результат отсортируйте по возрастанию id пользователя.

Поля в результирующей таблице: user_id, orders_count, orders_avg, orders_diff

Пояснение:

В этой задаче можно использовать подзапрос, написанный в первых заданиях этого урока. Чтобы не пришлось дважды писать один и тот же подзапрос, можно использовать оператор WITH.

##### ОТВЕТ:

In [ ]:
# Чужое
with 
    orders_users as (       
    select user_id, count (distinct order_id) orders_count
    from user_actions
    group by user_id)
select user_id, orders_count, round((select avg(orders_count) from orders_users),2) as orders_avg ,  (orders_count - round((select avg(orders_count) from orders_users),2)) as orders_diff
from orders_users
order by user_id


In [ ]:
# Из паринципа )) доделал
SELECT user_id,
    COUNT(DISTINCT(order_id)) AS orders_count,
   (SELECT ROUND(AVG(count_order),2) FROM (SELECT COUNT(DISTINCT(order_id)) AS count_order FROM user_actions GROUP BY user_id)gg) AS orders_avg,
    COUNT(DISTINCT(order_id)) - (SELECT ROUND(AVG(count_order),2) FROM (SELECT COUNT(DISTINCT(order_id)) AS count_order FROM user_actions GROUP BY user_id)gg) AS orders_diff
FROM user_actions
GROUP BY 1
ORDER BY 1

## Задача 9.
Перед тем как пойти дальше, предлагаем вам решить ещё несколько задач на подзапросы.

Задание:

Выведите id и содержимое 100 последних доставленных заказов из таблицы orders. Содержимым заказов считаются списки с id входящих в заказ товаров. Результат отсортируйте по возрастанию id заказа.

Поля в результирующей таблице: order_id, product_ids

Пояснение:

Обратите внимание, что содержимое заказов находится в таблице orders, а информация о действиях с заказами — в таблице courier_actions.



##### ОТВЕТ:

In [ ]:
SELECT order_id,
    product_ids
FROM orders
WHERE order_id in 
    (SELECT order_id
    FROM courier_actions
    WHERE action = 'deliver_order'
    ORDER BY time DESC 
    LIMIT 100)
ORDER BY order_id

## Задача 10.
Задание:

Из таблицы couriers выведите всю информацию о курьерах, которые в сентябре 2022 года доставили 30 и более заказов. Результат отсортируйте по возрастанию id курьера.

Поля в результирующей таблице: courier_id, birth_date, sex

Пояснение:

Обратите внимание, что информация о курьерах находится в таблице couriers, а информация о действиях с заказами — в таблице courier_actions.



##### ОТВЕТ:

In [ ]:
# МОЕ!
SELECT courier_id, birth_date, sex
FROM couriers
WHERE courier_id in 
    (SELECT courier_id
    FROM 
        (SELECT courier_id,
            COUNT(order_id) AS count_order
        FROM courier_actions
        WHERE (time BETWEEN '2022-09-01' AND '2022-09-30') 
        AND action = 'deliver_order'
        GROUP BY 1
        HAVING  COUNT(order_id) >29
        ) as2
    )
ORDER BY courier_id

In [ ]:
# Чужое
with cour as (select courier_id from courier_actions where action='deliver_order' 
              and EXTRACT(year from time) = 2022 and EXTRACT(month from time)=09 group by 1 HAVING count(*) >= 30)
select courier_id, birth_date, sex from couriers
where courier_id in (select courier_id from cour)
order by 1

## Задача 11.
В этой задаче совместим наши знания о конструкции CASE с подзапросами.

Задание:

Назначьте скидку 15% на товары, цена которых превышает среднюю цену на все товары на 50 и более рублей, а также скидку 10% на товары, цена которых ниже средней на 50 и более рублей. Цену остальных товаров внутри диапазона (среднее - 50; среднее + 50) оставьте без изменений. При расчёте средней цены, округлите её до двух знаков после запятой.

Выведите информацию о всех товарах с указанием старой и новой цены. Колонку с новой ценой назовите new_price. Результат отсортируйте сначала по убыванию прежней цены в колонке price, затем по возрастанию id товара.

Поля в результирующей таблице: product_id, name, price, new_price

##### ОТВЕТ:

In [ ]:
WITH avg_price AS(SELECT ROUND(AVG(price),2) FROM products)

SELECT product_id, 
            name, 
            price,
    CASE
        WHEN price - (SELECT * FROM avg_price) >= 50 THEN price * 0.85
        WHEN price - (SELECT * FROM avg_price) <= -50 THEN price * 0.9
        ELSE price
    END AS  new_price
   
FROM products
ORDER BY 3 DESC, 1

## Задача 12.
Давайте снова поработаем с массивами и освоим новую функцию unnest, которая пригодится нам в дальнейших задачах. Функция unnest предназначена для разворачивания массивов и превращения их в набор строк:

SELECT unnest(ARRAY['one','two','three'])

Результат:
one
two
three


В примере выше функция unnest превратила исходный список из трёх элементов в набор из трёх строк.

Если бы в исходной таблице помимо списка был столбец с каким-либо значением, то это значение автоматически проставилось бы напротив значений в каждой образовавшейся строке:

SELECT 'row', unnest(ARRAY['one','two','three'])

Результат:
row    one
row    two
row    three


А теперь рассмотрим работу функции unnest на реальном примере.

Задание:

Выберите все колонки из таблицы orders, но в качестве последней колонки укажите функцию unnest, применённую к колонке product_ids. Новую колонку назовите product_id. Выведите только первые 100 записей результирующей таблицы. Посмотрите на результат работы функции unnest и постарайтесь разобраться, что произошло с исходной таблицей.

Поля в результирующей таблице: creation_time, order_id, product_ids, product_id

##### ОТВЕТ:

In [ ]:
SELECT 
    creation_time,
    order_id,
    product_ids,
    UNNEST(product_ids) AS product_id
FROM orders
LIMIT 100

## Задача 13.
А теперь применим unnest для решения практической задачи.

Задание:

Используя функцию unnest, определите 10 самых популярных товаров в таблице orders. Самыми популярными будем считать те, которые встречались в заказах чаще всего. Если товар встречается в одном заказе несколько раз (т.е. было куплено несколько единиц товара), то это тоже учитывается при подсчёте. 

Выведите id товаров и сколько раз они встречались в заказах. Новую колонку с количеством покупок товара назовите times_purchased.

Поля в результирующей таблице: product_id, times_purchased

Пояснение:

В этом задании вам необходимо сначала развернуть списки с товарами в заказах, а затем для каждого товара посчитать, сколько раз он встретился в данных.  Для определения самых популярных товаров используйте оператор LIMIT. 

##### ОТВЕТ:

In [ ]:
SELECT product_id,
        COUNT(product_id) AS times_purchased
FROM (SELECT 
    order_id,
    UNNEST(product_ids) AS product_id
    FROM orders) qq
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10

## * Задача 14.
И напоследок ещё пара заданий со звёздочкой, чтобы точно убедиться, что мы разобрались с подзапросами.

Задание:

Из таблицы orders выведите id и содержимое заказов, которые включают хотя бы один из пяти самых дорогих товаров, доступных в нашем сервисе. Результат отсортируйте по возрастанию id заказа.

Поля в результирующей таблице: order_id, product_ids

Пояснение:

В этой задаче вам снова пригодится функция unnest. Также для упрощения кода можно использовать табличные выражения.

Если совсем не получается:

Попробуйте сначала определить 5 самых дорогих товаров, поместите эту таблицу в подзапрос, затем в отдельном подзапросе разверните списки товаров с помощью unnest, а потом уже в этой развёрнутой таблице просто отфильтруйте строки с теми заказами, которые содержат id товаров из списка самых дорогих. Так как в заказ могут войти сразу несколько самых дорогих товаров, проследите за тем, чтобы в результат не попали дубликаты заказов.

##### ОТВЕТ:

In [ ]:
# Чужое
WITH expensive_products AS 
 (SELECT product_id
   FROM products
  ORDER BY price DESC  
  LIMIT 5)
  
SELECT DISTINCT order_id, product_ids
  FROM (SELECT order_id, product_ids, unnest(product_ids) AS prod
          FROM orders) sub
 WHERE prod IN (SELECT * FROM expensive_products)
 ORDER BY order_id

In [ ]:
# МОЕ
SELECT DISTINCT order_id,
    product_ids
FROM (
    SELECT 
    order_id,
    product_ids,
    UNNEST(product_ids) AS  prod
    FROM orders
    )qq
    WHERE prod IN
        (SELECT product_id
        FROM products
        ORDER BY price DESC
        LIMIT 5
        )
    ORDER BY 1

## * Задача 15.
Задание:

Посчитайте возраст каждого пользователя в таблице users. Возраст измерьте числом полных лет, как мы делали в прошлых уроках. Возраст считайте относительно последней даты в таблице user_actions. В результат включите колонки с id пользователя и возрастом. Для тех пользователей, у которых в таблице users не указана дата рождения, укажите среднее значение возраста всех остальных пользователей, округлённое до целого числа. Колонку с возрастом назовите age. Результат отсортируйте по возрастанию id пользователя.

Поля в результирующей таблице: user_id, age

Пояснение:

В этой задаче вам придётся написать несколько подзапросов и, возможно, использовать табличные выражения. Пригодятся функции DATE_PART, AGE и COALESCE. Основная сложность заключается в заполнении пропусков средним значением — подумайте, как это можно сделать, и постройте запрос вокруг своего подхода. 

##### ОТВЕТ:

In [ ]:
WITH real_age AS (SELECT
                user_id,
                DATE_PART('year', AGE((SELECT DATE(MAX(time)) FROM user_actions), birth_date)) AS ages
                FROM users)
SELECT
    user_id,
    COALESCE(ages, ROUND((SELECT AVG(ages) FROM real_age)))  AS age
FROM
  real_age
ORDER BY 1

## Подведём итоги
В этом уроке мы:

Систематизировали накопленные знания и окончательно разобрались с порядком выполнения операторов в запросах.

Научились составлять подзапросы и узнали, что их можно применять в блоках SELECT, FROM, WHERE и HAVING.

Познакомились с табличными выражениями и оператором WITH, научились писать более аккуратный и понятный код.

Ещё немного поработали с датами, изучили функцию NOW и узнали, как проводить арифметические операции с интервалами.

Познакомились с новой функцией unnest и научились разворачивать списки со значениями в расширенные таблицы.

Решили несколько сложных задач на подзапросы и табличные выражения с применением всех полученных знаний.

Заметили, как сильно мы продвинулись в написании SQL-запросов всего за несколько уроков?

И тем не менее до сих пор мы были в какой-то степени ограничены структурой отдельных таблиц и почти никак не учитывали 
связи между ними.

Пришло время с этим разобраться — впереди нас ждёт ключевая тема, которая окончательно развяжет нам руки и позволит решать практически любые задачи, с которыми опытные аналитики сталкиваются в своей работе.